In [1]:
include("src/SpinonStructure.jl")
include("src/BZMath.jl")


using .BZmath
using .SpinonStructure
using LaTeXStrings
using StaticArrays
using SparseArrays
using LinearAlgebra
using Plots

In [2]:
using BenchmarkTools
using Profile

In [15]:
k_density_spinon_dispersion=60
k_density_specweight=20

G = @SMatrix [0. 0. 0.;
	 0. 0. 0.;
	 1. 0. 0.]


Jpm = 0.03
Bmin =sqrt(9*Jpm/5)
println("minimum B = ",Bmin)

lengths = [1,2,3,4,5]

sim_repo = [ SimulationParameters("$(Lx)",
        lattice=geom.PyroPrimitive(Lx,1,1),
        A=zeros(Lx,4),
        Jpm=-0.03,
        B=-0.4*[1,1,0]/√2,
        n_samples = 10000
    ) for Lx in lengths
    ];

minimum B = 0.23237900077244503


In [16]:
csim_repo = [CompiledModel(sim_repo[1])]
λ = csim_repo[1].lambda
for L in lengths[2:end]
    push!(csim_repo, CompiledModel(sim_repo[L], λ))
end

get_0flux_sim(Lx,Ly,Lz) = CompiledModel(
    SimulationParameters("$(Lx)",
        lattice=geom.PyroPrimitive(Lx,Ly,Lz),
        A=zeros(Lx*Ly*Lz,4),
        Jpm=-0.03,
        B=-0.4*[1,1,0]/√2,
        n_samples = 10000
    ),
    λ)

get_0flux_sim (generic function with 1 method)

In [5]:
const Vec3_F64 = SVector{3,Float64}

@inline function GreenFunction!(G, E, U, i, j)
    for l in eachindex(E)
        G[l] = conj(U[i, l]) * U[j,l] / (2*E[l])
    end
end

function my_corr_at(Q::Vec3_F64, p1::Vec3_F64, csim::CompiledModel)
  
    E1, U1 = spinon_dispersion( p1, csim)
    #p2 = geom.wrap_BZ(csim.sim.lat, p1+Q) # p1+Q
	p2 = p1+Q

    E2, U2 = spinon_dispersion( p2, csim)
    # Both p's must appear with the same sign, or else we break p-> p + delta 
    # invariance (required by gauge symmetry)

	
    QQ_tensor = SMatrix{3,3,Float64}(diagm([1.,1.,1.]) - Q*Q'/(Q'*Q))

	# preallocate everything
    f=length(csim.sim.lat.tetra_sites)
    S_pm = zeros(ComplexF64, f,f)
    
    delta_S_pm = zeros(ComplexF64, f,f)

    x1 = zeros(ComplexF64, f)
    x2 = zeros(ComplexF64, f)

    jB =0
    jpB=0

    A_sites = geom.A_sites(csim.sim.lat)
    
    for μ=1:4, ν=1:4
        for (jA, rA) in enumerate(A_sites), (jpA, rpA) in enumerate(A_sites) 
            jB = csim.nn_index_A[jA][μ]
            jpB = csim.nn_index_A[jpA][ν]

			# <S+ S->
            # the "l" bit
			GreenFunction!(x1, E1, U1, jA, jpA)
            @assert x1 == conj.(U1[jA, :]) .* U1[jpA, :] ./ (2*E1) 
   #          # the "l'" bit
			GreenFunction!(x2, E2, U2, jpB, jB)
            @assert x2 == U2[jB, :] .* conj.(U2[jpB, :]) ./ (2*E2)
 
            # delta_S_pm .= x1*transpose(x2)
            for l=1:f,lp=1:f
                    delta_S_pm[l,lp] = x1[l]*x2[lp]
            end
            @assert delta_S_pm == x1*transpose(x2)

            
            delta_S_pm .*= (
				exp(1im*(csim.sim.A[jA,μ]-csim.sim.A[jpA, ν]))*
				exp(1im* ( Q -2*p2)'* (geom.pyro[μ]-geom.pyro[ν])) 
				)
				
			S_pm .+= delta_S_pm
            
			
        end
    end
    E = [e1 + e2 for e1 in E1, e2 in E2]::Matrix{Float64}
    return E, S_pm
end



my_corr_at (generic function with 1 method)

In [6]:
Q= @SVector [1.,0.2,-0.2]
P= @SVector [-1.,-0.2,0.2]

@allocated corr_at(Q, P, csim_repo[1])

7845840

In [7]:
@allocated corr_at(Q, P, csim_repo[2])

30432

In [8]:
@benchmark corr_at(Q, P, csim_repo[4])


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  417.750 μs …  57.581 ms  ┊ GC (min … max): 0.00% … 98.74%
 Time  (median):     439.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   456.162 μs ± 626.142 μs  ┊ GC (mean ± σ):  1.36% ±  1.81%

         ▂▅█▆█▆▄▂                                                
  ▂▂▂▃▄▅▇█████████▇▆▆▅▅▄▄▅▄▄▄▃▄▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂ ▄
  418 μs           Histogram: frequency by time          524 μs <

 Memory estimate: 61.39 KiB, allocs estimate: 177.

In [9]:
@allocated spinon_dispersion(Q, csim_repo[4])

26992

In [10]:
csim = CompiledModel(SimulationParameters("test",
        lattice=geom.PyroPrimitive(2,3,4),
        A=zeros(2*3*4,4),
        Jpm=-0.03,
        B=-0.4*[1,1,0]/√2,
        n_samples = 10000
    ), λ)

corr_at(Q, P, csim)

([0.4773668603503127 0.48895572421059114 … 1.1079347174124352 1.1165237731477817; 0.49402979069217584 0.5056186545524542 … 1.1245976477542983 1.1331867034896448; … ; 1.1075041272197712 1.1190929910800498 … 1.738071984281894 1.7466610400172402; 1.115479917543099 1.1270687814033773 … 1.7460477746052212 1.7546368303405677], ComplexF64[-5.854691731421724e-18 + 1.734723475976807e-18im -9.974659986866641e-17 + 5.160802341031001e-17im … -6.946305442195704e-18 - 1.3349239248727773e-18im 4.228388472693467e-18 - 3.2526065174565133e-19im; 1.3791051634015616e-16 - 2.987743702417301e-17im 1.691355389077387e-17 + 1.000175675422205e-18im … 2.026857298453559 + 4.4165794675429746e-16im -8.944667923005412e-18 + 1.337469862687703e-18im; … ; 1.0950441942103595e-17 + 6.505213034913027e-19im 8.023096076392733e-18 - 5.421010862427522e-20im … -4.1186977060240354e-20 + 1.2149099441586837e-18im -2.7647155398380363e-18 - 5.149960319306146e-19im; -4.662069341687669e-18 + 2.2768245622195593e-18im 1.428301933350296

In [18]:
csim = get_0flux_sim(5,5,1)

@benchmark corr_at(Q, P, csim)



BenchmarkTools.Trial: 46 samples with 1 evaluation.
 Range (min … max):  108.069 ms … 160.427 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     108.821 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   109.920 ms ±   7.623 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅                                                             
  ██▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  108 ms        Histogram: log(frequency) by time        160 ms <

 Memory estimate: 837.81 KiB, allocs estimate: 192.

In [11]:
function test()
    X = zeros(1000)
end


test (generic function with 1 method)

In [12]:
@allocated test()

8192

In [13]:
8192÷8

1024